# Intro to agents



This session is an introduction to AI agents, covering their fundamental concepts, characteristics, and components of agentic LLM applications. It sets the stage for evaluating and building intelligent, autonomous systems.

What you will do:
- build your first AI agent using ADK and Gemini
- create a multi-agent system with specialized agents
- explore architectural patterns for multi-agent systems
By the end of this session, you will have a solid understanding of AI agents and practical experience in building and deploying them using modern tools and frameworks.



In [ ]:
!pip install google-adk

In [41]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

In [66]:
# import libraries specific to agentic AI
import pandas as pd

# import ADK components
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

In [43]:
# run in vs code only 
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("missing key")


In [44]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [52]:

# test that GOOGLE_API_KEY is set correctly
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="One sentence summary: Which chocolate is best for boosting creative work?",
)

print(response.text)

High-percentage dark chocolate (70% cocoa or more) is best for boosting creative work due to its rich flavonoids and stimulating compounds like theobromine, which enhance blood flow to the brain and improve focus.


In [ ]:
# do not run in VS code
# Setup for Colab

# set GOOGLE_API_KEY in Colab secrets
# the below retrieves the set key from the Colab
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# helper function to be used 

# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")


A Python variable (GOOGLE_API_KEY) lives only in your process/namespace and must be passed explicitly to callers.
`os.environ["GOOGLE_API_KEY"]` or  `os.getenv("GOOGLE_API_KEY")` places the value in the process environment so:
libraries that read `env vars `(or third‑party code) can find it without you passing it,
subprocesses inherit it,

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

### Components of an agent

In [61]:
# root agent
root_agent = Agent(
    name="Researcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    description="A research agent that assist with research tasks.",
    instruction="You are a research assistant, specialising in finance research. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent component.")

✅ Root Agent component.


### Tools

Agents have tools: in general, functions, to simplify some vendors provide in-built functions

Examples of tool categories for Gemini agents:

- Gemini API tools
- Google Cloud tools
- Third-party tools

Tool limitations

[Runner](https://google.github.io/adk-docs/runtime/runner/overview) is the central component that acts as the orchestrator.
It simplifies the process of creating agents that can perform various tasks by leveraging different tools.


In [62]:
# runner = orchestrator <- use "root agent"
# InMemoryRunner
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner.")

✅ Runner.


Simple demo


In [63]:
response = await runner.run_debug(
    "Locate the latest publication on Agentic AI in finance at http://arxiv.org/. Reply with an http link to the paper and one sentence summary"
)


 ### Created new session: debug_session_id

User > Locate the latest publication on Agentic AI in finance at http://arxiv.org/. Reply with an http link to the paper and one sentence summary
Researcher > A recent publication on Agentic AI in finance from arXiv, "The Agentic Regulator: Risks for AI in Finance and a Proposed Agent-based Framework for Governance," discusses how generative and agentic AI are rapidly entering financial markets, outpacing existing governance structures. The paper proposes a modular governance architecture composed of four layers of "regulatory blocks" to address the risks associated with these evolving AI technologies in financial systems.

Link: https://arxiv.org/abs/2512.11933


In [ ]:
Prompt -> LLM -> Text

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer